# Argo CD (GitOps Continuous Delivery for Kubernetes)

## Goals
- Understand what **Argo CD** is and how it implements **GitOps**.
- Know **why** teams use it for Kubernetes delivery (auditability, drift detection, repeatability).
- Learn key primitives: **Application**, **sync**, **health**, and **sync policies**.
- See a realistic **Application manifest pseudo-code** example.


## Prerequisites
- Kubernetes basics: namespaces, deployments, services, configmaps/secrets.
- Git basics and a simple repo layout for manifests/Helm/Kustomize.
- Basic CD concepts: environments, promotion, rollback.


## What is Argo CD?

**Argo CD** is a Kubernetes-native **continuous delivery** tool that follows the **GitOps** model.

GitOps idea (source of truth):
- Desired state lives in **Git** (Kubernetes manifests, Helm charts, Kustomize overlays).
- The cluster is continuously reconciled to match that desired state.

Argo CD runs inside (or connected to) your Kubernetes cluster and continuously compares:
- **Desired**: what Git says should exist
- **Live**: what is actually running in the cluster


## What is it used for (and why teams choose it)?

Common uses:
- Deploy apps to Kubernetes from Git (single cluster or many clusters).
- Manage environment overlays (dev/stage/prod) with clear diffs.
- Detect and optionally fix **configuration drift** (manual changes in-cluster).

Why teams use it:
- **Auditability**: Git history becomes the deployment history.
- **Repeatability**: recreate environments by applying the same Git state.
- **Separation of concerns**: CI builds artifacts; Argo CD handles delivery to Kubernetes.
- **Safety**: diffs are visible; rollbacks can be as simple as reverting a commit.


## How it works (mental model)

A useful picture:
```
Git repo (manifests/helm/kustomize)
        |
        |  (poll/webhook)
        v
     Argo CD
        |
        |  (compare desired vs live)
        v
Kubernetes API (apply/sync)  --->  Running resources
```

Key mechanics:
- Argo CD maintains an **Application** object that points to a Git source (repo/path/revision) and a destination (cluster/namespace).
- It computes a **diff** between Git and the live cluster.
- **Sync** applies the Git-defined resources to the cluster.
- **Health** checks indicate whether resources are running correctly (deployment ready, etc.).
- Optional **automated sync** keeps the cluster continuously reconciled.


## Core concepts (minimum you should know)
- **Application**: the unit Argo CD deploys; binds source (Git) to destination (cluster).
- **Sync**: bring live state to desired state (manual or automated).
- **Sync policy**: rules for automated sync, pruning, and self-healing.
- **Prune**: delete resources that exist in the cluster but were removed from Git.
- **Self-heal**: revert manual in-cluster changes back to Git-defined state.
- **Projects**: RBAC and boundaries (which repos/namespaces/clusters an app may use).
- **Sources**: raw manifests, Helm, Kustomize, or other supported generators.


## Typical patterns
- **GitOps split**: CI builds/pushes an image; CD updates Git (image tag/version) and Argo syncs.
- **App-of-apps**: a root application manages many child applications for platform bootstrapping.
- **Environment overlays**: `kustomize` overlays or separate Helm values per env.
- **Progressive delivery**: pair with Argo Rollouts for canary/blue-green (optional).


## Example: Argo CD Application manifest (pseudo-code)

This is a representative Application definition.

```yaml
# PSEUDO-CODE (argocd Application)
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: payments-api-staging
  namespace: argocd
spec:
  project: default

  source:
    repoURL: https://github.com/org/platform-deployments.git
    targetRevision: main
    path: apps/payments-api/overlays/staging
    # Alternatively, Helm:
    # chart: payments-api
    # helm:
    #   valueFiles: [values-staging.yaml]

  destination:
    server: https://kubernetes.default.svc
    namespace: payments

  syncPolicy:
    automated:
      prune: true      # remove resources deleted from Git
      selfHeal: true   # revert manual drift
    syncOptions:
      - CreateNamespace=true
      - PruneLast=true
```

A simple GitOps flow:
1) CI builds `payments-api` image and pushes `ghcr.io/org/payments-api:<sha>`.
2) A commit updates the deployment manifest in Git to reference the new image tag.
3) Argo CD detects the Git change and syncs it into the cluster.


## What Argo CD is capable of
- Continuous reconciliation (detect drift, show diffs, sync changes).
- Multi-app and multi-cluster management with RBAC boundaries.
- Declarative, auditable Kubernetes delivery with strong UX (UI/API/CLI).

Good fit when:
- Your deployment target is Kubernetes and you want Git to be the source of truth.

Less ideal when:
- Your deployments are not Kubernetes-based, or you need imperative orchestration that doesn't map to desired-state reconciliation.


## Pitfalls & quick tips
- Decide how you handle secrets: avoid committing raw secrets; consider sealed secrets / external secrets.
- Be explicit about ownership: manual `kubectl edit` fights GitOps and will be reverted if self-heal is enabled.
- Use Argo CD Projects + RBAC to restrict where apps can deploy.
- Be careful with `prune`: safe when Git is authoritative, risky if resources are created out-of-band.

## Exercises
- Create `dev` and `prod` overlays and describe what differs (replicas, resources, ingress).
- Enable automated sync for dev only; keep prod manual with approvals.
- Introduce a drift and observe how Argo CD reports it.

## References
- Argo CD Docs: https://argo-cd.readthedocs.io/
- GitOps principles: https://opengitops.dev/
- Argo Rollouts (optional): https://argo-rollouts.readthedocs.io/
